# Generate Training Set (Part 2)

These training sets compare long forms within each NSF.

## Setup

In [1]:
import pandas as pd
import numpy as np
import itertools
from fuzzywuzzy import fuzz

#### Load Dataset

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/lisavirginia/clinical-abbreviations/master/modules/Step2Output_Clinical_Abbreviation_Acronym_Crosswalk.csv',
                 sep='|',
                 header=0,
                 index_col=False,
                 na_filter=False,
                 dtype=object)

In [3]:
df.sample(3, random_state=0)

,GroupID,EntryID,SF,SFUI,NormSF,NSFUI,PrefSF,LF,LFUI,NormLF,PrefLF,Source,SFEUI,LFEUI,Type,Score,Count,Frequency,UMLS.CUI
33582,,E033583,α-GPDH,S103760,α_gpdh,N072157,,alpha-glycerophosphate dehydrogenase,L041419,alpha glycerophosphate dehydrogenase,,UMLS,E0412935,E0769306,acronym,,,,
70982,,E070983,PRPPs,S056625,prpps,N248835,,phosphoribosyl pyrophosphate synthetase,L128319,phosphoribosylpyrophosphate synthetase,,UMLS,E0571504,E0047486,acronym,,,,
192802,,E192803,IPH,S036667,iph,N193753,,intra-peritoneal haemorrhage,L097757,,,UMLS,E0703883,E0703882,acronym,,,,


## Initial Filtering

Identifies **most similar string** that is **not equivalent** within each **NSF**.

We used the standard Levenshtein distance similarity ratio available [here](https://github.com/seatgeek/fuzzywuzzy).

#### Use Normalized Long Form Where Possible

In [4]:
df['TrainLF'] = np.where(df['NormLF']=='', df['LF'], df['NormLF'])

In [5]:
df.sample(3, random_state=0)

,GroupID,EntryID,SF,SFUI,NormSF,NSFUI,PrefSF,LF,LFUI,NormLF,PrefLF,Source,SFEUI,LFEUI,Type,Score,Count,Frequency,UMLS.CUI,TrainLF
33582,,E033583,α-GPDH,S103760,α_gpdh,N072157,,alpha-glycerophosphate dehydrogenase,L041419,alpha glycerophosphate dehydrogenase,,UMLS,E0412935,E0769306,acronym,,,,,alpha glycerophosphate dehydrogenase
70982,,E070983,PRPPs,S056625,prpps,N248835,,phosphoribosyl pyrophosphate synthetase,L128319,phosphoribosylpyrophosphate synthetase,,UMLS,E0571504,E0047486,acronym,,,,,phosphoribosylpyrophosphate synthetase
192802,,E192803,IPH,S036667,iph,N193753,,intra-peritoneal haemorrhage,L097757,,,UMLS,E0703883,E0703882,acronym,,,,,intra-peritoneal haemorrhage


In [6]:
df=df.head(100)
df.shape

(100, 20)

#### Compute Similarities

In [7]:
df = df.sort_values(by=['NSFUI'])
df.head(10)

,GroupID,EntryID,SF,SFUI,NormSF,NSFUI,PrefSF,LF,LFUI,NormLF,PrefLF,Source,SFEUI,LFEUI,Type,Score,Count,Frequency,UMLS.CUI,TrainLF
69,,E000070,AAT,S003250,aat,N006062,,Aachen aphasia test,L003442,Aachen aphasia test,,UMLS,E0000051,E0500260,acronym,,,,,Aachen aphasia test
77,,E000078,AAT,S003250,aat,N006132,,Academic Aptitude test,L003503,Academic Aptitude test,,UMLS,E0000051,E0500261,acronym,,,,,Academic Aptitude test
90,,E000091,AB,S003282,ab,N006214,,Ace bandage,L003589,Ace bandage,,UMLS,E0000052,E0338802,acronym,,,,,Ace bandage
18,,E000019,AA,S003081,aa,N006781,,African American,L004037,AfricanAmerican,,UMLS,E0000048,E0420574,acronym,,,,,AfricanAmerican
87,,E000088,AAT,S003250,aat,N006795,,African animal trypanosomosis,L004047,African animal trypanosomosis,,UMLS,E0000051,E0515595,acronym,,,,,African animal trypanosomosis
19,,E000020,AA,S003081,aa,N006904,,African-American,L004091,AfricanAmerican,,UMLS,E0000048,E0420574,acronym,,,,,AfricanAmerican
93,,E000094,AB,S003282,ab,N007093,,Alcian blue,L004234,Alcian blue,,UMLS,E0000052,E0000202,acronym,,,,,Alcian blue
1,,E000002,AA,S003081,aa,N007108,,Alcoholics Anonymous,L004250,,,UMLS,E0000048,E0000204,acronym,,,,,Alcoholics Anonymous
60,,E000061,AAA,S003100,aaa,N007119,,Aleuria aurantia agglutinin,L004259,Aleuria aurantia agglutinin,,UMLS,E0000049,E0598120,acronym,,,,,Aleuria aurantia agglutinin
66,,E000067,AAMD,S003184,aamd,N007509,,American Association on Mental Deficiency,L004532,,,UMLS,E0000050,E0000277,acronym,,,,,American Association on Mental Deficiency


In [ ]:
train = []

In [ ]:
for NSFUI in df.NSFUI.unique():
    uniq_LFs = set(df.at[df['NSFUI'] == NSFUI]['TrainLF'].values)
    for pair in itertools.combinations(uniq_LFs, 2):
        distance = fuzz.ratio(pair[0], pair[1])
        if distance > 60:
            pair_dictionary = {'LF1':pair[0], 'LF2':pair[1], 'distance':distance} 
            train.append(pair_dictionary)

In [ ]:
train = pd.DataFrame(train)  

In [ ]:
train.sample(3, random_state=0)

## Random Subset

Select random subset of highly and moderately similar long forms for manual annotation.

In [ ]:
TrainSet3 = train[train.distance < 98]
TrainSet3 = TrainSet3.sample(frac=0.10)
TrainSet3 = TrainSet3.drop(columns=['distance'])
TrainSet3['Synonym'] = ''

In [ ]:
TrainSet1.sample(3, random_state=0)

#### Export

In [ ]:
TrainSet3.to_csv('TrainingSet3.csv',
                 index=False,
                 header=True,
                 sep='|')